# Predictive Data Analytics

In this notebook we will build an ML model and evaluate results

In [1]:
from pyspark.sql import SparkSession

# Add here your team number teamx
team = "team15"

# location of your Hive database in HDFS
warehouse = "project/hive/warehouse"

spark = SparkSession.builder\
        .appName("{} - spark ML".format(team))\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

#We can also add
# .config("spark.sql.catalogImplementation","hive")\ 
# But this is the default configuration
# You can switch to Spark Catalog by setting "in-memory" for "spark.sql.catalogImplementation"


In [2]:
spark.sql("SHOW DATABASES").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             root_db|
|     team0_projectdb|
|team12_hive_proje...|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
|    team21_projectdb|
|    team22_projectdb|
|    team23_projectdb|
|    team25_projectdb|
|    team26_projectdb|
|    team27_projectdb|
|    team28_projectdb|
|    team29_projectdb|
+--------------------+
only showing top 20 rows



In [3]:
spark.sql("USE team15_projectdb").show()

++
||
++
++



In [4]:
spark.sql("SHOW TABLES").show()

+----------------+--------------------+-----------+
|       namespace|           tableName|isTemporary|
+----------------+--------------------+-----------+
|team15_projectdb|     car_description|      false|
|team15_projectdb|car_vehicles_ext_...|      false|
|team15_projectdb|          q1_results|      false|
|team15_projectdb|          q2_results|      false|
|team15_projectdb|          q3_results|      false|
|team15_projectdb|          q4_results|      false|
|team15_projectdb|          q5_results|      false|
+----------------+--------------------+-----------+



In [5]:
spark.sql("SELECT * FROM team15_projectdb.car_vehicles_ext_part_bucket").show()

+----------+--------------------+-----+-----------------+-------------+--------------------+-------------+-----------+------+--------+------------+---------+---------+-----------+-----------+------------------+-------------------+--------+
|  entry_id|          region_url|price|manufactured_year| manufacturer|               model|car_condition|  cylinders|  fuel|odometer|transmission|car_drive| car_size|   car_type|paint_color|          latitude|          longitude|us_state|
+----------+--------------------+-----+-----------------+-------------+--------------------+-------------+-----------+------+--------+------------+---------+---------+-----------+-----------+------------------+-------------------+--------+
|7301824643|yubasutter.craigs...| 6250|             2006|          bmw|                325i|     like new|6 cylinders|   gas|  140000|   automatic|      rwd|  compact|      sedan|     silver| 39.17245864868164|-121.63050842285156|      ca|
|7304352956|yubasutter.craigs...|12995| 

In [6]:
print(*spark.catalog.listDatabases(), sep='\n')

Database(name='default', description='Default Hive database', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/apps/hive/warehouse')
Database(name='root_db', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/root/root_db')
Database(name='team0_projectdb', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/team0/project/hive/warehouse')
Database(name='team12_hive_projectdb', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/team12/project/hive/warehouse')
Database(name='team13_projectdb', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/team13/project/hive/warehouse')
Database(name='team14_projectdb', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/team14/project/hive/warehouse')
Database(name='team15_projectdb', description='', locationUri='hdfs://hadoop-02.uni.innopolis.ru:8020/user/team15/project/hive/warehouse')
Database(name='team16_projectdb', description='', loc

In [7]:
print(*spark.catalog.listTables("team15_projectdb"), sep='\n')

Table(name='car_description', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='car_vehicles_ext_part_bucket', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='q1_results', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='q2_results', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='q3_results', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='q4_results', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)
Table(name='q5_results', database='team15_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)


In [8]:
cars = spark.read.format("avro").table('team15_projectdb.car_vehicles_ext_part_bucket')

In [9]:
cars.printSchema()

root
 |-- entry_id: long (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: long (nullable = true)
 |-- manufactured_year: integer (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- car_condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: integer (nullable = true)
 |-- transmission: string (nullable = true)
 |-- car_drive: string (nullable = true)
 |-- car_size: string (nullable = true)
 |-- car_type: string (nullable = true)
 |-- paint_color: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- us_state: string (nullable = true)



In [10]:
lat_long = spark.sql("SELECT region_url, latitude, longitude FROM team15_projectdb.car_vehicles_ext_part_bucket").collect()

In [11]:
lat_long[:10]

[Row(region_url='quincy.craigslist.org', latitude=38.622398376464844, longitude=-92.54560089111328),
 Row(region_url='stlouis.craigslist.org', latitude=39.00120162963867, longitude=-90.96240234375),
 Row(region_url='stlouis.craigslist.org', latitude=38.56631851196289, longitude=-90.134521484375),
 Row(region_url='stlouis.craigslist.org', latitude=38.56633758544922, longitude=-90.1344985961914),
 Row(region_url='stlouis.craigslist.org', latitude=38.511253356933594, longitude=-90.43413543701172),
 Row(region_url='stlouis.craigslist.org', latitude=38.51133728027344, longitude=-90.43411254882812),
 Row(region_url='stlouis.craigslist.org', latitude=38.211814880371094, longitude=-90.0129165649414),
 Row(region_url='stlouis.craigslist.org', latitude=38.21134567260742, longitude=-90.01270294189453),
 Row(region_url='stlouis.craigslist.org', latitude=38.2116813659668, longitude=-90.01304626464844),
 Row(region_url='stlouis.craigslist.org', latitude=38.211612701416016, longitude=-90.012962341308

In [12]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters, HasInputCols, HasOutputCols
from pyspark.sql.types import ArrayType, DoubleType, FloatType
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
import pyproj
import numpy as np


class LatLongToXYZ(Transformer, HasInputCols, HasOutputCols):
    @keyword_only
    def __init__(self, inputCols=None, outputCols=None):
        super(LatLongToXYZ, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    def _transform(self, dataset):
        def f(lat, lon):
            lat = np.deg2rad(lat)
            lon = np.deg2rad(lon)
            alt = np.deg2rad(0)
            
            A = 6378137  # WGS-84 semi-major axis
            E2 = 6.6943799901377997e-3  # WGS-84 first eccentricity squared
            n = A / np.sqrt(1 - E2 * np.sin(lat) * np.sin(lat))
            
            x = (n + alt) * np.cos(lat) * np.cos(lon)
            y = (n + alt) * np.cos(lat) * np.sin(lon)
            z = (n * (1 - E2) + alt) * np.sin(lat)

            return [float(x), float(y), float(z)]

        # make an udf
        t = ArrayType(FloatType())
        udf_func = udf(f, t)

        # get calculations
        in_cols = dataset.select(self.getInputCols()).columns

        #  Apply the UDF to the input columns
        dataset = dataset.withColumn(self.getOutputCols()[0], udf_func(*in_cols)[0])
        dataset = dataset.withColumn(self.getOutputCols()[1], udf_func(*in_cols)[1])
        dataset = dataset.withColumn(self.getOutputCols()[2], udf_func(*in_cols)[2])

        return dataset

    @keyword_only
    def setParams(self, inputCols=None, outputCols=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

In [13]:
# dataframe size
print((cars.count(), len(cars.columns)))

# scema (dtypes actually schema is df.shema)
print(cars.dtypes)

# show first 10
print(cars.show(10))

# select unique
print(cars.select('paint_color').distinct().show())

(81866, 18)
[('entry_id', 'bigint'), ('region_url', 'string'), ('price', 'bigint'), ('manufactured_year', 'int'), ('manufacturer', 'string'), ('model', 'string'), ('car_condition', 'string'), ('cylinders', 'string'), ('fuel', 'string'), ('odometer', 'int'), ('transmission', 'string'), ('car_drive', 'string'), ('car_size', 'string'), ('car_type', 'string'), ('paint_color', 'string'), ('latitude', 'double'), ('longitude', 'double'), ('us_state', 'string')]
+----------+--------------------+-----+-----------------+------------+--------------------+-------------+-----------+------+--------+------------+---------+---------+--------+-----------+------------------+------------------+--------+
|  entry_id|          region_url|price|manufactured_year|manufacturer|               model|car_condition|  cylinders|  fuel|odometer|transmission|car_drive| car_size|car_type|paint_color|          latitude|         longitude|us_state|
+----------+--------------------+-----+-----------------+------------+-

In [14]:
# as we can see, our data is messed up (cooper s. == cooper s && countryman == cooper countryman && others...)
cars.select('model').distinct().where(cars.manufacturer == 'mini').collect()  # this trend is the same for other manufacturers as well

[Row(model='clubman cooper hatchback'),
 Row(model='cooper clubman'),
 Row(model='cooper countryman base'),
 Row(model='cooper countryman s all4awd'),
 Row(model='cooper hardtop'),
 Row(model='cooper mt convertible'),
 Row(model='cooper s countryman'),
 Row(model='cooper s countryman all 4'),
 Row(model='cooper s hardtop'),
 Row(model='cooper s hatchback'),
 Row(model='cooper s jcw'),
 Row(model='cooper s.'),
 Row(model='cooper sport'),
 Row(model='coopers s countryman'),
 Row(model='hardtop cooper hatchback'),
 Row(model='john cooper works'),
 Row(model='cargo'),
 Row(model='coooper s camden'),
 Row(model='cooper'),
 Row(model='cooper 2dr coupe base'),
 Row(model='cooper 6 spd convertible'),
 Row(model='cooper base 2dr hatchback'),
 Row(model='cooper clubman jcw'),
 Row(model='cooper clubman s'),
 Row(model='cooper convertible'),
 Row(model='cooper country'),
 Row(model='cooper countryman'),
 Row(model='cooper countryman all4'),
 Row(model='cooper countryman s'),
 Row(model='cooper co

In [68]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.sql.functions import col

# determine columns to use
drop_cols = ['entry_id', 'model', 'paint_color']  # model is too variable in general
categorical_cols = ['manufacturer', 'car_condition', 'cylinders', 'fuel', 'transmission', 'car_drive', 'car_size', 'car_type', 'us_state', 'region_url']
numerical_cols = ['manufactured_year', 'odometer']
location_cols = ['latitude', 'longitude']

# make indexer
stages = []

for cat_col in categorical_cols:
    string_indexer = StringIndexer(inputCol=cat_col, outputCol=f"{cat_col}_index")
    encoder = OneHotEncoder(
        inputCols=[f"{cat_col}_index"],
        outputCols=[f"{cat_col}_encoded"]
    )
    stages += [string_indexer, encoder]

# location transformer
latlong_transformer = LatLongToXYZ(inputCols=location_cols, outputCols=['x', 'y', 'z'])
stages.append(latlong_transformer)

# concatinate all encoded data
assembler = VectorAssembler(
    inputCols=[f"{cat_col}_encoded" for cat_col in categorical_cols] + numerical_cols + ['x', 'y', 'z'],
    outputCol="features"
)
stages += [assembler]

In [69]:
# make a full pipeline
pipeline = Pipeline(stages=stages)
# pipeline.write().overwrite().save("pipeline")

In [70]:
# make final adjustements & process data
cars = cars.dropna()

data_preprocessor = pipeline.fit(cars)
data = data_preprocessor.transform(cars).select('features', 'price')

In [71]:
data.select('features').take(1)

[Row(features=SparseVector(542, {2: 1.0, 41: 1.0, 46: 1.0, 52: 1.0, 56: 1.0, 59: 1.0, 62: 1.0, 70: 1.0, 120: 1.0, 247: 1.0, 537: 1995.0, 538: 199200.0, 539: 1229148.625, 540: -4787495.0, 541: 4017578.25}))]

In [72]:
# from pyspark.ml.feature import VectorIndexer
# from pyspark.ml.evaluation import RegressionEvaluator


# # select non-categorical features & transform them
# feature_indexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
# transformed = feature_indexer.transform(data)

# # Display the output Spark DataFrame
# transformed.show()

In [73]:
(train_data, test_data) = data.randomSplit([0.8, 0.2])

In [74]:
# from pyspark.sql.functions import percentile

# data.select(
#     percentile("price", [0.25, 0.5, 0.75], lit(1)).alias("quantiles")
# ).show()

In [75]:
from pyspark.ml.regression import LinearRegression

# Create Linear Regression Model & final pipeline
lr = LinearRegression(labelCol="price", featuresCol='features')

In [76]:
train_data.count()

65595

In [77]:
# train model
model = lr.fit(train_data)

In [78]:
# make predictions
predictions = model.transform(test_data)

# show predictions
predictions.show()

+--------------------+-----+-------------------+
|            features|price|         prediction|
+--------------------+-----+-------------------+
|(542,[0,40,45,52,...|31500|-219449.97382525107|
|(542,[0,40,45,52,...|33499|-189159.41529206318|
|(542,[0,40,45,52,...|39900| -188977.3125532193|
|(542,[0,40,45,52,...|27900|  671367.9657976518|
|(542,[0,40,45,52,...|22600|   592799.012605165|
|(542,[0,40,45,52,...| 7999| -304088.9328949542|
|(542,[0,40,45,52,...|43900|-267371.25804451376|
|(542,[0,40,45,52,...| 7400|-283122.29327716446|
|(542,[0,40,45,52,...| 7400|-283122.29327716446|
|(542,[0,40,45,52,...|18900|-181959.52914901407|
|(542,[0,40,45,52,...|18999|  520966.7431972601|
|(542,[0,40,45,52,...|10500| -266252.9751788009|
|(542,[0,40,46,52,...| 4900|-242080.80913911774|
|(542,[0,40,46,52,...|12900|  -280948.992060779|
|(542,[0,40,46,52,...|19999| -319193.2481292379|
|(542,[0,40,46,52,...|11550| -221646.3110006626|
|(542,[0,40,46,52,...| 7900| -308908.0478423039|
|(542,[0,40,46,52,..

In [79]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse))

Root Mean Squared Error (RMSE) on test data = 912292.845491784


In [80]:
from pyspark.ml.regression import DecisionTreeRegressor

# Create Linear Regression Model & final pipeline
dt = DecisionTreeRegressor(labelCol="price", featuresCol='features')

In [81]:
model = dt.fit(train_data)

In [82]:
# make predictions
predictions = model.transform(test_data)

# show predictions
predictions.show()

+--------------------+-----+------------------+
|            features|price|        prediction|
+--------------------+-----+------------------+
|(542,[0,40,45,52,...|49900|22305.717811704835|
|(542,[0,40,45,52,...| 7495|10052.525326830868|
|(542,[0,40,45,52,...| 7495|10052.525326830868|
|(542,[0,40,46,52,...| 7480|10052.525326830868|
|(542,[0,40,47,52,...|10999|10052.525326830868|
|(542,[0,40,47,52,...| 2800|10052.525326830868|
|(542,[0,40,47,52,...| 9500|10052.525326830868|
|(542,[0,40,47,53,...|48900|22305.717811704835|
|(542,[0,40,47,53,...|18000|10052.525326830868|
|(542,[0,41,45,52,...|    0|22305.717811704835|
|(542,[0,41,45,52,...|    0|22305.717811704835|
|(542,[0,41,46,52,...| 2995|10052.525326830868|
|(542,[0,41,47,52,...|    0|22305.717811704835|
|(542,[0,41,47,52,...| 4500|10052.525326830868|
|(542,[0,41,47,53,...|12900|10052.525326830868|
|(542,[0,41,47,53,...| 6000|10052.525326830868|
|(542,[0,41,47,53,...| 5000|10052.525326830868|
|(542,[0,42,45,52,...|21800|22305.717811

In [83]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse))

Root Mean Squared Error (RMSE) on test data = 125696.52159756508
